In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
import random
import numpy as np
from tqdm.notebook import tqdm
from tabulate import tabulate
from omnibelt import load_json, save_json, load_yaml, save_yaml
from pathlib import Path
from collections import Counter
import requests
import json
import clipboard
# from googlefinance import getQuotes
from bs4 import BeautifulSoup
# import requests, sys, lxml.html

from fuzzywuzzy import fuzz

from yahooquery import Ticker

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

from src import *

C:\Users\anwan\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# EO 13959 text: https://home.treasury.gov/system/files/126/13959.pdf

In [3]:
raw = '''Aero Engine Corp of China
Aviation Industry Corporation of China, Ltd. (AVIC)
China Academy of Launch Vehicle Technology (CALT)
China Aerospace Science & Technology Corporation (CASC)
China Aerospace Science & Industry Corporation (CASIC)
China Communication Construction Group Company, Ltd.
China Electronics Corporation (CEC)
China Electronics Technology Group Corporation (CETC)
China Mobile Communications
China National Chemical Corporation (ChernChina)
China National Chemical Engineering Group Co., Ltd. (CNCEC)
China National Nuclear
China Nuclear Engineering & Construction Corporation (CNECC)
China General Nuclear Power
China Railway Construction Corporation (CRCC)
China Shipbuilding Industry Corporation (CSIC)
China South Industries Group Corporation (CSGC)
China Spacesat
China State Construction Group Co., Ltd.
China State Shipbuilding Corporation (CSSC)
China Telecommunications
China Three Gorges Corporation Limited
China United Network Communications Group Co Ltd
CRRC Corporation
Dawning Information Industry Co. (Sugon)
Hikvision
Huawei
Inspur Group
Norinco
Panda Electronics
Sinochem Group Co Ltd'''
blacklist = raw.split('\n')

In [4]:
etfs = load('ETF', 'holdings', pbar=tqdm)
etfs.keys()

dict_keys(['XITK', 'LIT', 'EWY', 'IGOV', 'VXF', 'ICLN', 'FNCL', 'VONG', 'FLJP', 'QQQ', 'ICVT', 'IAU', 'CNYA', 'MCHI', 'IWV', 'SCHP', 'SKYY', 'ACES', 'FHLC', 'VGK', 'SOCL', 'CIBR', 'ESPO', 'EMXC', 'FMAT', 'IBB', 'SPY', 'VTI', 'EWC', 'LQD', 'KWEB', 'EWG', 'IXUS', 'PHO', 'IAGG', 'ASHR', 'ARKK', 'SPDW', 'FIDU', 'DIA', 'KOMP'])

In [5]:
china = {}
for tk, holdings in etfs.items():
    interest = [row for row in holdings if row[2] is not None and 'China' in row[2]]
    if len(interest):
        china[tk] = interest
china.keys()

dict_keys(['XITK', 'LIT', 'VXF', 'ICLN', 'VONG', 'QQQ', 'ICVT', 'CNYA', 'MCHI', 'IWV', 'SKYY', 'SOCL', 'ESPO', 'EMXC', 'IBB', 'VTI', 'KWEB', 'IXUS', 'IAGG', 'ASHR', 'ARKK', 'SPDW', 'KOMP'])

In [13]:
matches = []

done = set()

threshold = 70

for i, (tk, holdings) in enumerate(china.items()):
    if tk not in done:
        todo = tqdm(holdings)
        for _, name, _, q in todo:
            todo.set_description(f'{tk} ({i+1}/{len(china)}) : {name}')
            for cmpy in blacklist:
#                 match = fuzz.token_set_ratio(cmpy, name)
                match = fuzz.token_sort_ratio(cmpy, name)
                if match > threshold:
                    matches.append([tk, name, cmpy, match])

#             if len(matches):
#                 break

#         if len(matches):
#             print(f'Found {len(matches)} matches')
#             break
    
#     done.add(tk)

# matches

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/114 [00:00<?, ?it/s]

  0%|          | 0/299 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

In [16]:
print(tabulate(sorted(matches, key=lambda row: row[-1], reverse=True), headers=['Ticker', 'Holding', 'Blacklist', 'Match']))

Ticker    Holding                                              Blacklist                                                       Match
--------  ---------------------------------------------------  ------------------------------------------------------------  -------
CNYA      China United Network Communications Ltd              China United Network Communications Group Co Ltd                   90
MCHI      China United Network Communications Ltd              China United Network Communications Group Co Ltd                   90
IXUS      China United Network Communications Ltd              China United Network Communications Group Co Ltd                   90
ASHR      China United Network Communications Ltd              China United Network Communications Group Co Ltd                   90
CNYA      China National Chemical Engineering Co Ltd           China National Chemical Engineering Group Co., Ltd. (CNCEC)        88
MCHI      China National Chemical Engineering Co Ltd           China 

In [14]:
# print(tabulate(sorted(matches, key=lambda row: row[-1], reverse=True), headers=['Ticker', 'Name', 'Blacklist', 'Match']))

In [15]:
# matches